In [ ]:
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import gfile

from collections import Counter
from itertools import combinations

In [ ]:
## Lees csv-files in
csv_path = 'csv_files/'
mid_to_label_new = pd.read_csv(str(csv_path) + 'class_labels_indices_amsterdam.csv', quotechar = '"')
balanced_train_segments = pd.read_csv(str(csv_path) + 'balanced_train_segments.csv', sep = ";")
unbalanced_train_segments = pd.read_csv(str(csv_path) + 'unbalanced_train_segments.csv', sep = ";")

In [ ]:
## Verkrijg alle labels en labelcombinaties (uit csv-file)
def getLabelOccurences(segments_file):
    
    ## Houd labels bij en label-combinaties (elke combi uit een example)
    labels = []
    label_combinations = []

    # for each row
    for i in range(segments_file.shape[0]):

        # labels uit de example
        label_combination = []
        mids = segments_file.loc[i, 'positive_labels'].split(",")
        
        # for each m-id in a string
        for mid in mids:
            new_label = mid_to_label_new.loc[mid_to_label_new['mid']==str(mid), 'index']
            
            if(new_label.shape[0]>0):
                labels.extend(new_label)
                new_label_combi = str(new_label.values[0])
                label_combination.extend(new_label_combi)
    
        if len(label_combination)>0:
            label_combinations.append(label_combination)
    
    return(labels, label_combinations)

In [ ]:
## Kies of je de balanced of unbalanced set wilt lezen
#bal_labels, bal_label_combinations = getLabelOccurences(balanced_train_segments)
unbal_labels, unbal_label_combinations = getLabelOccurences(unbalanced_train_segments)

In [ ]:
## Houd totale aantal bij
sum_occur = 0
all_labels = unbal_labels # bal_labels

## Laat # voorkomens per label zien
cnt_labels = Counter(all_labels)
for i in cnt_labels.most_common(len(all_labels)):
    sum_occur = sum_occur + i[1]
    print(mid_to_label_new.loc[np.int(i[0])]['display_name'], ' : ', str(i[1]))

In [ ]:
label_combinations = unbal_label_combinations

## Bereken aantal voorkomens per combinatie
## Tellen van lijsten in lijsten is lastig, vandaar dat ik er strings van maak: [0,4] wordt ["0_4"]
str_combis = []

for combi in label_combinations:
    str_combi = [str(i) for i in combi]
    str_combi = "_".join(str_combi)
    str_combis.append(str_combi)

In [ ]:
## Geef meest voorkomende combinaties

cnt_label_combinations = Counter(str_combis)
for label_combination in cnt_label_combinations.most_common(20):
    label_names = []
    label_list = label_combination[0].split("_")
    for i in label_list:
        label_names.append(str(mid_to_label_new.loc[np.int(i)]['display_name']))
    label_names = ', '.join(label_names)
    print(label_names,': ', label_combination[1])

In [ ]:
## Code die kolom toevoegt aan label-csv (proportion) en deze in de forloop vult

mid_to_label_new["proportion"] = 0

for i in range(mid_to_label_new.shape[0]):
    nr_occur = cnt_labels[i]
    mid_to_label_new.loc[i,"proportion"] = round(nr_occur/sum_occur,8)

In [ ]:
## Schrijf bestandje weg
mid_to_label_new.to_csv(str(csv_path) + "class_labels_indices_proportions.csv", sep = ";")

In [ ]:
###############################################################################
####
###############################################################################
#### Hieronder staat andere code (deze code telt de labels vanuit tf-records)

In [ ]:
## Folder met tf-records
train_data_pattern = 'tfrecords/train_all_data_new_temp/*.tfrecord'
test_data_pattern = 'tfrecords/test_data_new/*.tfrecord'

In [ ]:
## Hieronder worden labels geteld door het lezen van tf-records
all_labels = []
label_combinations = []

In [ ]:
def getLabelsFromTfRecord(tfrecords_filename):
    
    ## Iterator that goes through all examples of tf-record
    record_iterator = tf.python_io.tf_record_iterator(path=tfrecords_filename)
    
    for string_record in record_iterator:
        
        ## Take one example
        example = tf.train.SequenceExample()
        example.ParseFromString(string_record)
 
        ## Get labels
        new_labels = example.context.feature['labels'].int64_list.value
       
        for label in new_labels:
            all_labels.append(int(label))
        label_combinations.append(new_labels)

In [ ]:
## Labels uit tf-records
files = gfile.Glob(train_data_pattern)

for file in files:
   
    getLabelsFromTfRecord(file)

In [ ]:
## Houd totale aantal bij
sum_occur = 0

## Laat # voorkomens per label zien
cnt_labels = Counter(all_labels)
for i in cnt_labels.most_common(len(all_labels)):
    sum_occur = sum_occur + i[1]
    print(mid_to_label_new.loc[np.int(i[0])]['display_name'], ' : ', str(i[1]))

In [ ]:
#label_combinations = getLabelCombinations(balanced_train_segments)
#label_combinations = getLabelCombinations(unbalanced_train_segments)

## Bereken aantal voorkomens per combinatie
## Tellen van lijsten in lijsten is lastig, vandaar dat ik er strings van maak: [0,4] wordt ["0_4"]
str_combis = []

for combi in label_combinations:
    str_combi = [str(i) for i in combi]
    str_combi = "_".join(str_combi)
    str_combis.append(str_combi)

In [ ]:
## Geef meest voorkomende combinaties

cnt_label_combinations = Counter(str_combis)
for label_combination in cnt_label_combinations.most_common(20):
    label_names = []
    label_list = label_combination[0].split("_")
    for i in label_list:
        label_names.append(str(mid_to_label_new.loc[np.int(i)]['display_name']))
    label_names = ', '.join(label_names)
    print(label_names,': ', label_combination[1])

In [ ]:
sum_occur